In [1]:
import torch
import torch.nn as nn
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets

In [2]:
hn = 3
class Model(nn.Module):
     
    def __init__(self):
        super().__init__()
        self.stepLength = 1.2
        self.hidden = torch.zeros((1, hn))
        
        self.Whh = nn.Linear(hn, hn)
        self.Whx = nn.Linear(1, hn, bias = False)
        
        self.layer1 = nn.Linear(1 + hn, 64)
        self.layer2 = nn.Linear(64, 32)
        self.layer3 = nn.Linear(32, 8)
        self.layer4 = nn.Linear(8, 1)
      
    
    def forward_RNN(self, x):
        self.hidden = torch.tanh(self.Whh(self.hidden) + self.Whx(x))
        
        
    def forward_DNN(self, x):
        
        y = torch.cat((self.hidden, x), 1)
        
        y = torch.tanh(self.layer1(y))
        y = torch.tanh(self.layer2(y))
        y = torch.tanh(self.layer3(y))
        y = self.layer4(y)
     
        return y*torch.exp(-0.1*x**2)
    
    def sample(self, N):
        total = 0
        x = torch.Tensor(4*np.random.random((N,1)) - 2)
        psi_old = self.forward_DNN(x)
        
        for i in range(10):
            x_new = x + 1.5*torch.Tensor(2*np.random.random((N,1)) - 1)
            psi_new = self.forward_DNN(x_new)
            
            bool_array = (psi_new/psi_old)**2 > torch.Tensor(np.random.random((N,1)))
            x[bool_array] = x_new[bool_array]
            psi_old[bool_array] = psi_new[bool_array]
            total += 1
            
        return x, total
    
    def resetHidden(self, N):
        self.hidden = torch.zeros((N, hn))
        self.forward_RNN(torch.zeros((N, 1)))
        

In [3]:
torch.manual_seed(42)
np.random.seed(42)
model = Model()
optimizer = torch.optim.Adam(model.parameters())

### Using naive minimization of energy

In [4]:
epochs = 100
N = 1000
for epoch in tqdm(range(epochs)): 
    psi_total = 1

    model.resetHidden(N) 
    x1 = model.sample(N)[0].detach().requires_grad_()
    psi1 = model.forward_DNN(x1)

    model.forward_RNN(x1)
    x2 = model.sample(N)[0].detach().requires_grad_()
    psi2 = model.forward_DNN(x2)

    psi_total = psi1*psi2
    dfdx   = [torch.autograd.grad(a, x1, create_graph=True)[0][i] for i, a in enumerate(psi_total)]
    d2fdx2 = [torch.autograd.grad(a, x1, create_graph=True)[0][i] for i, a in enumerate(dfdx)]

    kinetic1 = -0.5*torch.Tensor(d2fdx2)/psi_total
    potential1 = 0.5*x1**2

    dfdx   = [torch.autograd.grad(a, x2, create_graph=True)[0][i] for i, a in enumerate(psi_total)]
    d2fdx2 = [torch.autograd.grad(a, x2, create_graph=True)[0][i] for i, a in enumerate(dfdx)]

    kinetic2 = -0.5*torch.Tensor(d2fdx2)/psi_total
    potential2 = 0.5*x2**2

    E_L = (kinetic1 + potential1 + kinetic2 + potential2 + 1/torch.abs(x1-x2)).detach()
    
    PE = torch.mean(psi_total/psi_total.detach()*E_L)
    P  = torch.mean(psi_total/psi_total.detach())
    E  = torch.mean(E_L)   

    
    loss = 2*(PE - P*E)
    
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    
    grad = 0
    for param in model.parameters():
            grad += torch.sum(param.grad**2)

    
    if epoch % 10 == 0:
        print(f"epoch: {epoch}, Grad: {grad}, Energy: {E.item()}")

  1%|          | 1/100 [00:10<16:52, 10.22s/it]

epoch: 0, Grad: 580.4075927734375, Energy: 6.806632995605469


 11%|█         | 11/100 [01:48<14:38,  9.87s/it]

epoch: 10, Grad: 177.06956481933594, Energy: 5.639733791351318


 13%|█▎        | 13/100 [02:14<14:58, 10.33s/it]


KeyboardInterrupt: 

In [ ]:
def f(x):
    x_lin = torch.linspace(-5, 5, 100).reshape(100,-1)

    model.resetHidden(100)
    psi1 = model.forward_DNN(x_lin)[:,0].detach().numpy()

    x = torch.Tensor([[x]])

    model.forward_RNN(x)
    psi2 = model.forward_DNN(x_lin)[:,0].detach().numpy()


    plt.plot(x_lin[:,0], psi1**2/np.sum(psi1**2))
    plt.plot(x_lin[:,0], psi2**2/np.sum(psi2**2))
    plt.plot(x, 0.01, "o")
    plt.xlim((-5,5))
    plt.ylim((0, 0.05))

    plt.show()

interact(f, x=(-4.0, 4., 0.1));

In [ ]:
x_lin = torch.linspace(-4, 4, 100).reshape(100,-1)
psi = model.forward(x_lin)[:,0].detach().numpy()

plt.plot(x_lin[:,0], psi**2)
plt.plot(x_lin[:,0], 1/np.sqrt(np.pi)*np.exp(-x_lin[:,0]**2), "--")
plt.legend(["DNN", "analytical"])
plt.show()


## Estimating energy

In [ ]:
N = 100

E = 0
for i in tqdm(range(N)):
    model.resetHidden() 
    x1 = model.sample()[0].detach().requires_grad_()
    psi1 = model.forward_DNN(x1)

    model.forward_RNN(x1)
    x2 = model.sample()[0].detach().requires_grad_()
    psi2 = model.forward_DNN(x2)

    psi_total = psi1*psi2
    dfdx, = torch.autograd.grad(psi_total, x1, create_graph=True)
    d2fdx2, = torch.autograd.grad(dfdx, x1, create_graph=True)

    kinetic1 = -0.5*d2fdx2/psi_total
    potential1 = 0.5*x1**2

    dfdx, = torch.autograd.grad(psi_total, x2, create_graph=True)
    d2fdx2, = torch.autograd.grad(dfdx, x2, create_graph=True)

    kinetic2 = -0.5*d2fdx2/psi_total
    potential2 = 0.5*x2**2

    E += (kinetic1 + potential1 + kinetic2 + potential2 + 1/torch.abs(x1-x2)).detach()

E = E/N

print(E.item())

In [ ]:
model.sample()

### Added penalization of small amplitude

In [ ]:
torch.manual_seed(42)
np.random.seed(42)
model = Model()
optimizer = torch.optim.Adam(model.parameters())

In [ ]:
epochs = 200
N = 50
mu = 0.5
for epoch in tqdm(range(epochs)):
    PE_acc = 0
    P_acc = 0
    E_acc = 0
    grad = 0
    psi_pen = 0
    
    for i in range(N):
        x = model.sample()[0].detach().requires_grad_()
     
        psi = model.forward(x)
        psi_pen += (1 - psi**2)**2
        dfdx, = torch.autograd.grad(psi, x, create_graph=True)
        d2fdx2, = torch.autograd.grad(dfdx, x, create_graph=True)
        
        kinetic = -0.5*d2fdx2/psi
        potential = 0.5*x**2
        
        E_L = (kinetic + potential).detach()
        PE_acc += psi/psi.detach()*E_L
        P_acc  += psi/psi.detach()
        E_acc  += E_L   
    
    PE_acc /= N
    P_acc  /= N
    E_acc  /= N
    psi_pen /= N
    
    E = 2*(PE_acc - P_acc*E_acc) + mu*psi_pen
    
    optimizer.zero_grad()
    E.backward()
    optimizer.step()
    
    for param in model.parameters():
            grad += torch.sum(param.grad**2)

    
    if (epoch%10 == 0):
        print(f"epoch: {epoch}, Grad: {grad}, Energy: {E_acc.item()}")

## Testing auto grad

In [6]:
x = [torch.Tensor([3]).requires_grad_(), torch.Tensor([6]).requires_grad_()]
y = torch.Tensor([4, 8]).requires_grad_()

f = torch.Tensor(x)**3*y**2

print(f)
#f.backward()
#print(x.grad, y.grad)

dfdx = [torch.autograd.grad(a, b, create_graph=True)[0] for a, b in zip(f, x)]
print(dfdx)

d2fdx2 = [torch.autograd.grad(a, b, create_graph=True)[0] for a, b in enumerate(dfdx, x)]

print(d2fdx2)

#d2fdx2.backward()
#print(x.grad, y.grad)

#print(dfdx)


#ddfdxdx, = torch.autograd.grad(dfdx, x, create_graph=True)
#print(ddfdxdx.grad_fn(x)[1])

tensor([  432., 13824.], grad_fn=<MulBackward0>)


RuntimeError: One of the differentiated Tensors appears to not have been used in the graph. Set allow_unused=True if this is the desired behavior.